# Scaling and vector addition


In [6]:
%cd drive/My Drive/learning/data_science/coding_the_matrix/resources/

[Errno 2] No such file or directory: 'drive/My Drive/learning/data_science/coding_the_matrix/resources/'
/content


In [7]:
# Copyright 2013 Philip N. Klein
"""
This file contains a simple plotting interface, which uses an Iframe to
present a plot of points represented as either complex numbers or 2-vectors.
"""
from numbers import Number
from IPython.display import IFrame,HTML, display
import tempfile
import os
import atexit
from io import StringIO


def plot(L, scale=4, dot_size = 3):
    """ plot takes a list of points, optionally a scale (relative to a 200x200 frame),
        optionally a dot size (diameter) in pixels.It produces a string with 
        SVG representing the given plot, and opens the file in an Iframe. It returns nothing.
    """
    scalar = 200./scale
    origin = (210, 210)
    out_str = StringIO()
    out_str.write(
            """<html>
            <head>
            <title>plot</title>
            </head>
            <body>
            <svg height="450" width="450" xmlns="http://www.w3.org/2000/svg">\n'
            <line x1="0" y1="210" x2="420" y2="210"
            style="stroke:rgb(0,0,0);stroke-width:2"/>
            <line x1="210" y1="0" x2="210" y2="420"
            style="stroke:rgb(0,0,0);stroke-width:2"/>""")
    parsedImgPts = list()
    for pt in L:
      if isinstance(pt, Number):
        x,y = pt.real, pt.imag
      else:
          if isinstance(pt, tuple) or isinstance(pt, list):
            x,y = pt
          else:
            raise ValueError
      parsedImgPts.append((origin[0]+scalar*x, origin[1]-scalar*y)) 
    for (a, b) in parsedImgPts:       
        out_str.write('<circle cx="%d" cy="%d" r="%d" fill="red"/>\n'
                          % (a,b , dot_size))
    out_str.write("""</svg>\n</body>\n</html>""")
    display(HTML(out_str.getvalue()))
    out_str.close()
        

In [8]:
def add(v,w):
  return [v[0] + w[0], v[1] + w[1]]
def addn(v,w):
  return [v[i] + w[i] for i in range(len(v))]  


In [9]:
L = [(2,2), (3,2), (1.75,1), (2,1), (2.25,1), (2.5,1), (2.75,1), (3,1), (3.25,1)]
plot(L)

In [10]:
def scalar_vector_mult(alpha, v):
  return [alpha * x for x in v]

In [11]:
v = [3,2]
plot([scalar_vector_mult(i/10,v) for i in range(11)], 5)

In [12]:
plot([scalar_vector_mult(i/100,v) for i in range(100)] , 5)

# Dictionary representaion

In [13]:
"""setter operator"""
def setitems(v, d, val): v.f[d] = val

"""
getter
procedure getitem(v, d) with below spec
input: an instance of v of Vec. and an element d of the set v.D
output: the value of entry d of v"""
def getitems(v , d): return v.f[d] if d in v.f else 0

""""
procedure zero_vec(D) with the following spec:
input: set D
output: an instance of Vec representing a D-vector all of whose entries have value zero 
"""
def zero_vec(D): {d:0 for d in D }

"""equal operator overloading"""
def equal(u , v ):
  "compares the right hand element to the left hand element"
  assert u.D == v.D
  return all(getitems(u,key) == getitems(v,key) for key in v.f)

"""add operator overloading"""
def add(u,v):
  assert u.D == v.D
  s=u.copy()
  for k in u.D:
    setitems(s,k,getitems(u,k)+getitems(v,k)) 
  return s

"""dot operation"""
def dot(u,v):
  assert u.D == v.D
  s=0
  for k in u.D:
    s+= getitems(u,k)*getitems(v,k)
  return s

"""negative operation"""
def neg(v):
    u=v.copy()
    for k in v.D:
        setitems(u,k, -1*getitems(v,k))
    return u

"""scalar multiplication"""   
def scalar_mul(v, alpha):
    u=v.copy()
    for k in v.D:
      setitems(u,k, alpha*getitems(v,k)) 
    return u

class Vec:
    def __init__(self, labels, function):
        self.D = labels
        self.f = function

    __getitem__ = getitems
    __setitem__ = setitems
    __neg__ = neg
    __rmul__ = scalar_mul 

    def __mul__(self,other):
      if isinstance(other, Vec):
          return dot(self,other)
      else:
          return NotImplemented 

    __add__ = add

    def __radd__(self, other):
        "Hack to allow sum(...) to work with vectors"
        if other == 0:
            return self

    def __sub__(a,b):
        return a+(-b)

    __eq__ = equal

    def __hash__(self):
        h = hash(frozenset(self.D))
        for k,v in sorted(self.f.items(), key = lambda x:repr(x[0])):
            if v != 0:
                h = hash((h, hash(v)))
        return h

    def __repr__(self):
        return "Vec(" + self.D + "," + self.f + ")"

    def copy(self):
        return Vec(self.D, self.f.copy())

procedure list2vec(L)
input: a list L of field elements
output: an instance of V of Vec with domain {0,1,2...len(L) - 1} such that v[i] = L[i] for each integer i in the domain

In [14]:
def list2vec(L: list)-> Vec:
  return (Vec(set(rangelen((L)))), {k:x for k,x in enumerate(L)})

assertions

In [15]:
v = Vec({'a','b','c', 'd'},{'a':2,'c':1,'d':3})
assert v['d'] == 3
assert v['b'] == 0

In [16]:
v = Vec({'a', 'b', 'c'}, {'b':0})
v['b'] = 5
assert v['b'] == 5
v['a'] = 1
assert v['a'] == 1
v['a'] = 0
assert v['a'] == 0

In [17]:
assert Vec({'a', 'b', 'c'}, {'a':0}) == Vec({'a', 'b', 'c'}, {'b':0})
assert Vec({'a', 'b', 'c'}, {'a': 0}) == Vec({'a', 'b', 'c'}, {})
assert Vec({'a', 'b', 'c'}, {}) == Vec({'a', 'b', 'c'}, {'a': 0})

In [18]:
a = Vec({'a','e','i','o','u'}, {'a':0,'e':1,'i':2})
b = Vec({'a','e','i','o','u'}, {'o':4,'u':7})
c = Vec({'a','e','i','o','u'}, {'a':0,'e':1,'i':2,'o':4,'u':7})
assert a + b == c

a == Vec({'a','e','i','o','u'}, {'a':0,'e':1,'i':2})
b == Vec({'a','e','i','o','u'}, {'o':4,'u':7})
d = Vec({'x','y','z'}, {'x':2,'y':1})
e = Vec({'x','y','z'}, {'z':4,'y':-1})
f = Vec({'x','y','z'}, {'x':2,'y':0,'z':4})
assert d + e == f
  
d == Vec({'x','y','z'}, {'x':2,'y':1})
e == Vec({'x','y','z'}, {'z':4,'y':-1})
assert b + Vec({'a','e','i','o','u'}, {}) == b


In [19]:
u1 = Vec({'a','b'}, {'a':1, 'b':2})
u2 = Vec({'a','b'}, {'b':2, 'a':1})
assert u1*u2 == 5

assert u1 == Vec({'a','b'}, {'a':1, 'b':2})
assert u2 == Vec({'a','b'}, {'b':2, 'a':1})

v1 = Vec({'p','q','r','s'}, {'p':2,'s':3,'q':-1,'r':0})
v2 = Vec({'p','q','r','s'}, {'p':-2,'r':5})
assert v1*v2 == -4
    
w1 = Vec({'a','b','c'}, {'a':2,'b':3,'c':4})
w2 = Vec({'a','b','c'}, {'a':12,'b':8,'c':6})
assert  w1*w2 == 72

"""
The pairwise products should not be collected in a set before summing
    because a set eliminates duplicates
"""
v1 = Vec({1, 2}, {1 : 3, 2 : 6})
v2 = Vec({1, 2}, {1 : 2, 2 : 1})
assert v1 * v2 == 12

In [20]:
zero = Vec({'x','y','z','w'}, {})
u = Vec({'x','y','z','w'},{'x':1,'y':2,'z':3,'w':4})
assert 0*u == zero

assert 1*u == u
assert 0.5*u == Vec({'x','y','z','w'},{'x':0.5,'y':1,'z':1.5,'w':2})
assert u == Vec({'x','y','z','w'},{'x':1,'y':2,'z':3,'w':4})


In [21]:
u = Vec({1,3,5,7},{1:1,3:2,5:3,7:4})
assert -u == Vec({1, 3, 5, 7},{1: -1, 3: -2, 5: -3, 7: -4})
assert u == Vec({1,3,5,7},{1:1,3:2,5:3,7:4})
assert -Vec({'a','b','c'}, {'a':1}) == Vec({'a','b','c'}, {'a':-1})

# dot product

procedure list.dot(u, v)  with spec
input: equal length lists u and v of field elements
output: the dot product of u and v interpreted as vectors

In [22]:
def list_dot(u, v): return sum(u[i] * v[i] for i in range(len(u)))

# solving triangular equations

In [25]:
def triangular_solve(rowlist , b):
  x = zero_vec(rowlist[0].D)
  for i in reversed(range(len(rowlist))):
    x[i] = (b[i - rowslist[i] * b])/ rowlist[i][i]
  return x